In [1]:
import json
import pandas as pd
import math
import os
import xmltodict
import traceback
from slpp import slpp as lua
from compress_lua_table import CompressLuaTable as compress

In [ ]:
class bcolors:
    OK = '\033[92m' #GREEN
    WARNING = '\033[93m' #YELLOW
    FAIL = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR

In [ ]:
types = {}
_types = {}
heads = set()
# with open('SkillEffectTable.json', 'rb') as f:
#     data = json.load(f)
#     for field in data['Fields']:
#         types[field['FieldName']] = field
#         _types[field['FieldName']] = field['FieldTypeName']
#         heads.add(field['FieldName'])

# set(_types.values())
dir = 'E:\WorkSpace\ROGame-dev\config\Table\Configs'
for file in os.listdir(dir):
    with open(os.path.join(dir, file), 'rb') as file:
        data = json.load(file)
        for item in data['TableLocations']:
            print(item['ExcelPath'])

In [ ]:
data = pd.read_csv('SkillEffectTable.csv').drop([0])
columns = list(set(data.columns.tolist()) - heads)
data = data.drop(columns=columns)
lua_raw_data = data.to_dict('index')

def check_default(_v, cast, default):
    def is_nan(__v):
        if type(__v) == float:
            return math.isnan(__v)
    return default if is_nan(_v) else cast(_v)


def sequence_to_dict(value, partten, length, cast):
    if value is None or value.strip() == '':
        return value
    array = {}
    sequence = value.split(partten)
    if len(sequence) == length:
        for i in range(length):
            array[i + 1] = cast(sequence[i])
        return array
    return value


def vector_to_list(value, partten, cast, func=None, *args):
    if value is None or value.strip() == '':
        return value
    l = []
    sequence = value.split(partten)
    for s in sequence:
        if func is not None:
            l.append(func(s, *args))
        elif s.strip() != '':
            l.append(cast(s))
    return l


def iteritems_recursive(d):
    t = {}
    for k, v in d.items():
        if isinstance(v, dict):
            t[k] = iteritems_recursive(v)
        else:
            _type = _types[k]
            default = types[k]['DefaultValue']
            if _type == 'string':
                t[k] = check_default(v, str, default)
            elif _type == 'int':
                t[k] = check_default(v, int, default)
            elif _type == 'float':
                t[k] = check_default(v, float, default)
            elif _type == 'bool':
                t[k] = 'false' if v == 0 or v == 'nan' else 'true'
            elif _type == 'Sequence<int, 2>':
                t[k] = sequence_to_dict(check_default(v, str, default), '=', 2, int)
            elif _type == 'vector<Sequence<int, 2>>':
                args = ['=', 2, int]
                t[k] = vector_to_list(check_default(v, str, default), '|', int, sequence_to_dict, *args)
            elif _type == 'vector<Sequence<int, 3>>':
                args = ['=', 3, int]
                t[k] = vector_to_list(check_default(v, str, default), '|', int, sequence_to_dict, *args)
            elif _type == 'vector<int>':
                t[k] = vector_to_list(check_default(v, str, default), '|', int)
            elif _type == 'vector<float>':
                t[k] = vector_to_list(check_default(v, str, default), '|', float)
            elif _type == 'vector<string>':
                t[k] = vector_to_list(check_default(v, str, default), '|', str)
            elif _type == 'vector<vector<int>>':
                args = ['=', int]
                t[k] = vector_to_list(check_default(v, str, default), '|', int, vector_to_list, *args)
            else:
                print(f'`{bcolors.FAIL}' + _type + f'{bcolors.RESET}` is not processed!')
                # return
    return t

table = iteritems_recursive(lua_raw_data)
with open('SkillTable.lua', 'w', encoding='utf-8') as f:
    f.write('return ' + lua.encode(table))

In [ ]:
def is_int(n):
    try:
        int(n)
    except ValueError:
        return False
    return True

def is_float(n):
    try:
        float(n)
    except ValueError:
        return False
    return True

def is_boolean(n):
    if n == 'true' or n == 'false':
        return True
    return False

def to_boolean(n):
    if n == 'true': return True
    elif n == 'false': return False

def filter_key(k):
    return k == '@xmlns:xsd' or k == '@xmlns:xsi'

def iter_xml_recursive(d):
    t = {}
    for k, v in d.items():
        if isinstance(v, dict):
            t[k] = iter_xml_recursive(v)
        elif isinstance(v, list):
            t[k] = iter_xml_recursive({i : v[i] for i in range(len(v))})
        else:
            if v is None:
                t[k] = None
            elif is_int(v):
                t[k] = int(v)
            elif is_float(v):
                t[k] = float(v)
            elif is_boolean(v):
                t[k] = to_boolean(v)
            elif not filter_key(k):
                t[k] = v
                # return
    return t

with open('E:\WorkSpace\ROGame-dev\config\Assets\Resources\SkillData\Activity_XieNengRuQin\Monster_ShenGuan_ShengGuangFaZhen.txt', 'rb') as f:
    table = iter_xml_recursive(dict(xmltodict.parse(f)))
    # print(json.loads(json.dumps(xmltodict.parse(f))))

In [ ]:
from compress_lua_table import CompressLuaTable as compress
compress.process_file(table, 'Monster_ShenGuan_ShengGuangFaZhen')

In [ ]:
dir = 'E:\WorkSpace\ROGame-dev\config\Assets\Resources\SkillData'
EDITOR_DIR = './SkillData/'
if not os.path.exists(EDITOR_DIR):
    os.mkdir(EDITOR_DIR)
for path, dirs, fs in os.walk(dir):
    for f in fs:
        if f.endswith('.txt'):
            # print(f'processing {bcolors.OK}' + os.path.join(path, f) + f'{bcolors.RESET}...')
            # print(os.path.join(EDITOR_DIR + path[len(dir) + 1:], f.replace('.txt', '.lua')))
            _dir = os.path.join(EDITOR_DIR + path[len(dir) + 1:])
            if not os.path.exists(_dir):
                os.mkdir(_dir)
            with open(os.path.join(path, f), 'rb') as file:
                try:
                    table = iter_xml_recursive(dict(xmltodict.parse(file)))
                    with open(os.path.join(_dir, f.replace('.txt', '.lua')), 'w', encoding='utf-8') as w:
                        w.write('local ' + f[:-4] + ' = ' + lua.encode(table) + '\n return ' + f[:-4])
                except Exception as e:
                    print(f'{bcolors.FAIL} error: ' + os.path.join(path, f) + f'{bcolors.RESET}')
                    traceback.print_exc()


In [ ]:
import re

str = 'vector<string>'
str2 = 'vector<Sequence<float, 4>>'
pattern = re.compile('([1-9]\d*|string)')
type_pattern = 'Sequence|vector'
print(pattern.findall(str))
print(pattern.match(str2))
print(re.match('vector<[a-z]*>', str))
print(re.match('vector<(Sequence|vector)<[a-z]*, [1-9]\d*>>$', str2))
print(os.cpu_count())
re.match('_size=[1-9]\d*', '_size=1')
re.match('_t=[s|v]', '_t=v')

In [ ]:
%run main.py --f client --csv

In [ ]:
import subprocess

_dir = 'Table-client/'
for file in os.listdir(_dir):
    func = 'optimizer(\"{0}\", \"{1}\")'.format(_dir + file[:-4], file[:-4])
    try:
        result = subprocess.check_output(['lua53', '-l', 'DataTableOptimizer', '-e', func])
    except Exception as e :
        print(file)

In [6]:
import subprocess
import traceback


_dir = 'table-server'
# _dir = 'table-client'
for file in os.listdir('./' + _dir):
    func = '_load("{0}")'.format(_dir + '.' + file[:-4])
    try:
        # print(func)
        result = subprocess.check_output(['lua', '-l', 'test', '-e', func])
    except Exception as e :
        print(file)
        # pass
        # traceback.print_exc()
    # result = subprocess.check_output(['lua', '-l', 'test', '-e', func])
    

In [ ]:
import re
import sys
from numbers import Number
from slpp import slpp as lua
import six
import json


tab = ''
newline = ''
depth = 0

def __encode(obj):
    s = ''
    global depth
    global newline
    global tab
    if isinstance(obj, str):
        s += '"%s"' % obj.replace(r'"', r'\"')
    elif six.PY2 and isinstance(obj, unicode):
        s += '"%s"' % obj.encode('utf-8').replace(r'"', r'\"')
    elif six.PY3 and isinstance(obj, bytes):
        s += '"{}"'.format(''.join(r'\x{:02x}'.format(c) for c in obj))
    elif isinstance(obj, bool):
        s += str(obj).lower()
    elif obj is None:
        # pass
        s += 'nil'
    elif isinstance(obj, Number):
        s += str(obj)
    elif isinstance(obj, dict):
        depth += 1
        s += "{"
        key_list = ['%s' for k in obj.keys()]
        contents = [(key + '%s') % (k, __encode(v)) for (k, v), key in zip(obj.items(), key_list)]
        s += (',%s' % newline).join(contents)
        depth -= 1
        s += "}"
    return s

def encode(obj):
    s = ''
    if isinstance(obj, str):
        s += '"%s"' % obj.replace(r'"', r'\"')
    elif isinstance(obj, bool):
        s += str(obj).lower()
    elif obj is None:
        s += 'nil'
    elif isinstance(obj, Number):
        s += str(obj)
    elif isinstance(obj, dict):
        s += "{"
        contents = [encode(v) for _, v in obj.items()]
        s += ','.join(contents)
        s += "}"
    return s


def compress_lua(obj, name):
    s = 'local {} = {\n'.format(name)
    for _, value in obj.items():
        line = '\t{'
        for k, items in value.items():
            line += encode(items) + ','
        line = line[:-1] + '},\n'
        s += line

    # define default table
    s += '}\n\nlocal __default_table = {'
    for key, _ in indices.items():
        s += key + ','
    s = s[:-1] + '}\n'

    # add postfix

    return s

    
with open('./Table-client/AwardPackTable.lua', 'r') as f:
    table_name = 'AwardPackTable'
    text = f.read()
    obj = lua.decode(text)
    res = compress_lua(obj)
    res = 'local {} = '.format(table_name) + res
    # res = __encode(obj)
    with open('test.lua', 'w', encoding='utf-8') as w:
        w.write(res)
        w.write("\ndo\n")
        w.write("\tlocal base = {__index = __default_table, __newindex = function() error(\"Attempt to modify read-only table\") end}\n")
        w.write("\tfor k, v in pairs(%s) do\n" % (table_name))
        w.write("\t\tsetmetatable(v, base)\n")
        w.write("\tend\n")
        w.write("\tbase.__metatable = false\n")
        w.write("end\n")
        w.write("\nreturn %s\n" % (table_name))


In [ ]:
import re
from CppHeaderParser import CppHeader, CppParseError


class bcolors:
    OK = '\033[92m' #GREEN
    WARNING = '\033[93m' #YELLOW
    FAIL = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR


def filter_include(lines):
    res = ''
    for line in lines:
        if line.strip().startswith('$'): res += line.replace('$', '')
        else: res += line
    return res


def pascal_to_snake(camel: str):
    snake = re.sub(r'(?P<key>[A-Z])', r'_\g<key>',camel)
    return snake.lower().strip('_')


def parse_cpp(name, header):
    namespaces = set()
    classes = {}
    output_namespaces = 4 * ' ' + '// {}\n'.format(name)
    output_classes = 4 * ' ' + '// classes\n'
    output_variables = 4 * ' ' + '// variables\n'
    output_methods = 4 * ' ' + '// methods\n'
    output_enums = 4 * ' ' + '// enums\n'

    # variables & methods
    for class_name in header.classes.keys():
        # print(header.classes[class_name])
        namespace = header.classes[class_name]['namespace']
        namespaces.add(namespace)
        classes[class_name] = namespace
        
        for method in header.classes[class_name]['methods']['private']:
            output_methods += 4 * ' ' + '{}_proxy[\"{}\"] = &{}::{}::{};\n'.format(pascal_to_snake(class_name), method['name'], namespace, class_name, method['name'])

        for property in header.classes[class_name]['properties']['public']:
            output_variables += 4 * ' ' + '{}_proxy[\"{}\"] = &{}::{}::{};\n'.format(pascal_to_snake(class_name), property['name'], namespace, class_name, property['name'])

        for property in header.classes[class_name]['properties']['private']:
            output_variables += 4 * ' ' + '{}_proxy[\"{}\"] = &{}::{}::{};\n'.format(pascal_to_snake(class_name), property['name'], namespace, class_name, property['name'])

    # namespace
    for namespace in namespaces:
        output_namespaces += 4 * ' ' + 'sol::table {} = l.create_named_table(\"{}\");\n'.format(pascal_to_snake(namespace), namespace)

    # classes
    for class_, namespace in classes.items():
        output_classes += 4 * ' ' + 'auto {}_proxy = {}.new_usertype<{}::{}>(\"{}\");\n'.format(pascal_to_snake(class_), pascal_to_snake(namespace), namespace, class_, class_)

    # enums
    for enum in header.enums:
        s = ''
        enum_str = 8 * ' ' + '\"{}\", {}::{}::{}, \n'
        namespace = enum['namespace'][:-2]
        for variable in enum['values']:
            s += enum_str.format(variable['name'], namespace, enum['name'], variable['name'])
        # print(s[:-1])
        output_enums += 4 * ' ' + '{}.new_enum(\"{}\",\n{}\n);\n'.format(pascal_to_snake(namespace), enum['name'], s[:-1])

    print(output_namespaces, output_classes, output_methods, output_variables, output_enums)


with open('.config', 'r') as f:
    lines = f.readlines()
    dir_config = lines[2].strip()
    _dir = dir_config.split('#')[1]
    for file in os.listdir(_dir):
        if file.endswith('.pkg'):
            if file != 'buff_def.pkg': continue
            with open(os.path.join(_dir, file), 'r', encoding='utf-8') as pkg:
                content = filter_include(pkg.readlines())
                try:
                    cpp_header = CppHeader(content, argType='string', encoding='utf-8')
                    parse_cpp(file, cpp_header)
                except CppParseError as e:
                    # print(e)
                    print(f'{bcolors.FAIL}' + file + f'{bcolors.RESET}')

In [ ]:
from csv_to_lua import CSVToLua as csv

csv.set_writ_flag(2)
csv.setConfig('client', is_need_key=False, is_need_index=True, is_save_string=True)
csv.csv_to_lua()

global_string = csv.get_global_string()


In [10]:
def revert_dict(d):
    result = {}
    for k in d:
        if d[k] not in result:
            result[d[k]] = set()
        result[d[k]].add(k)
    return {k: result[k] if len(result[k]) > 1 else result[k].pop() for k in result}
def one_line(index):
    if index > LOCAL_TABLE_MAX: return REPEAT_KEY_PREFIX + '[' + str(index - LOCAL_TABLE_MAX) + ']'
    else: return REPEAT_KEY_PREFIX + str(index)

REPEAT_KEY_PREFIX = '__rt'
LOCAL_TABLE_MAX = 160
lines = []
index = 1
reversed_globel_string = revert_dict(global_string)
# print(reversed_globel_string)
for v, indices in reversed_globel_string.items():
    if isinstance(indices, set):
        lines.append('local {}={}\n'.format(one_line(index), v))
        index += 1
index = 1
for idx, s in global_string.items():
    if isinstance(reversed_globel_string[s], set): lines.append('str[{}] = {}'.format(idx, one_line(index)))
    lines.append('str[{}] = "{}"'.format(idx, s))

lines

In [51]:
import re
import pandas as pd

_dir = 'E:\WorkSpace\ROGame-dev\clientproj\Assets\Scripts\Lua'
modules = {}
for path, dirs, fs in os.walk(_dir):
    for f in fs:
        if f.endswith('.lua'):
            with open(os.path.join(path, f), 'r', encoding='utf-8') as file:
                pattern = re.compile('module\("\w+(\.\w+){0,1}", package.seeall\)')
                # pattern = re.compile('^module')
                s = ''.join([l for l in file])
                result = pattern.search(s)
                module_name_pattern = re.compile('"\w+(\.\w+){0,1}"')
                if result:
                    module_name = module_name_pattern.search(result.group()).group()
                    module_name = module_name[1:-1]
                    if module_name not in modules:
                        modules[module_name] = []
                    modules[module_name].append(f)
                    

modules = formatter_data = pd.DataFrame(modules.items(), columns=['Module Name', 'Files'])
modules.to_csv('modules.csv')

In [32]:
import pandas as pd
import re
from peel_deprecated_module import PeelDeprecatedModule as pdm
from rich.console import Console

console = Console()
pdm.peel()
modules = dict(sorted(pdm.get_modules().items()))
pattern = re.compile('(?P<key>\w+)')
unique_module = set()
for g, _ in modules.items():
    res = pattern.findall(g)
    if res and res[0] not in unique_module:
        unique_module.add(res[0])
        print('\n{} = {{}}\ndeclareGlobal("{}", {})'.format(res[0], res[0], res[0]))

    if res and len(res) == 2:
        print('{}.{} = {{}}'.format(res[0], res[1]))
# formatter_data = pd.DataFrame(modules.items(), columns=['Module Name', 'Files'])


ActivityTask = {}
declareGlobal("ActivityTask", ActivityTask)

Co = {}
declareGlobal("Co", Co)

Command = {}
declareGlobal("Command", Command)

CommandMacro = {}
declareGlobal("CommandMacro", CommandMacro)

Common = {}
declareGlobal("Common", Common)
Common.UI_TemplatePool = {}
Common.Utils = {}

CommonUI = {}
declareGlobal("CommonUI", CommonUI)
CommonUI.ModelAlarm = {}

Data = {}
declareGlobal("Data", Data)

Event = {}
declareGlobal("Event", Event)
Event.EventHelper = {}

EventConst = {}
declareGlobal("EventConst", EventConst)

MUIEvent = {}
declareGlobal("MUIEvent", MUIEvent)

ModuleData = {}
declareGlobal("ModuleData", ModuleData)
ModuleData.AuthData = {}
ModuleData.LandingAwardData = {}
ModuleData.RoleTagData = {}
ModuleData.SceneEnterData = {}
ModuleData.StallData = {}
ModuleData.SuitData = {}
ModuleData.TipsData = {}

ModuleMgr = {}
declareGlobal("ModuleMgr", ModuleMgr)
ModuleMgr.AccessoryAssistGuideMgr = {}
ModuleMgr.AlbumMgr = {}
ModuleMgr.AnnounceMgr = {}
ModuleMgr.BGMHouseMg